# 🛡️ ScamShield Family v8.1
## AI 기반 가족 보호형 피싱·스캠 예방 플랫폼

### v8.1 핵심 개선
- ✅ **100점 만점**: 직관적인 점수 체계 (AI 30점 + 메타 70점)
- ✅ **통계 기반 가중치**: 금융감독원/KISA 통계 기반
- ✅ **나눔고딕 폰트**: 한글 출력 안정화
- ✅ **실시간 스코어링**: 메타 정보 기반 컨텍스트 판단
- ✅ **Multi-Guardian**: 보호자 알림 시스템

---
## 1. 환경 설정

In [ ]:
!pip install -q transformers datasets torch scikit-learn pandas matplotlib seaborn accelerate

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONIOENCODING'] = 'utf-8'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Optional
import random
import re
import platform

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset as HFDataset

# ========================================
# 한글 폰트 설정 - 나눔고딕 우선
# ========================================
def setup_korean_font():
    """한글 폰트 설정 (나눔고딕 우선)"""
    # 나눔고딕 경로 후보
    nanum_paths = [
        '/Library/Fonts/NanumGothic.ttf',
        '/Library/Fonts/NanumGothic.otf',
        os.path.expanduser('~/Library/Fonts/NanumGothic.ttf'),
        os.path.expanduser('~/Library/Fonts/NanumGothic.otf'),
        '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
    ]
    
    # 나눔고딕 찾기
    for path in nanum_paths:
        if os.path.exists(path):
            fm.fontManager.addfont(path)
            font_name = fm.FontProperties(fname=path).get_name()
            plt.rcParams['font.family'] = font_name
            plt.rcParams['axes.unicode_minus'] = False
            return font_name
    
    # 시스템에서 한글 폰트 검색
    korean_fonts = [f.name for f in fm.fontManager.ttflist 
                    if any(x in f.name.lower() for x in ['nanum', 'gothic', 'gulim', 'dotum'])]
    if korean_fonts:
        plt.rcParams['font.family'] = korean_fonts[0]
        plt.rcParams['axes.unicode_minus'] = False
        return korean_fonts[0]
    
    return 'Default (한글 깨질 수 있음)'

font_name = setup_korean_font()
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ Device: {device}')
print(f'✅ 한글 폰트: {font_name}')

---
## 2. 메타 정보 스코어링 시스템 (100점 만점)

### 점수 체계
- **AI 모델 점수**: 최대 30점 (피싱 확률 기반)
- **메타 스코어**: 최대 70점 (발신자/URL/키워드 등)
- **합계**: 100점 만점

### 가중치 근거 (통계 기반)
| 항목 | 점수 | 근거 |
|------|------|------|
| 모르는 번호 | 18점 | 피싱 80% 미등록 발신 (금융감독원 2023) |
| 연락처 미등록 | 15점 | 미등록 시 피해 4.2배 증가 (Kim et al. 2024) |
| 첫 연락 | 10점 | 첫 연락 사기 78% (KISA 2023) |
| URL 포함 | 12점 | 스팸 67% URL 포함 (KISA 2023) |
| 전화번호 포함 | 7점 | 콜백 유도 43% |
| 금융 키워드 2+ | 10점 | 피싱 92% 금융키워드 포함 |
| 긴급 키워드 2+ | 6점 | 긴급성 호소 빈도 |

In [ ]:
class MetaScorer:
    """
    메타 정보 기반 위험도 스코어링 시스템 v8.1
    
    총점: 70점 만점 (AI 30점 + 메타 70점 = 100점)
    
    참고 자료:
    - 금융감독원 보이스피싱 피해 통계 (2023): 피해자 80%가 미등록 번호에서 발신
    - KISA 스팸 동향 분석 보고서 (2023): URL 포함 67%, 첫 연락 사기 78%
    - Kim et al. (2024) "Detection of Korean Phishing Messages": 미등록 시 피해 4.2배
    """
    
    # 100점 만점 기준 가중치 (메타 최대 70점)
    WEIGHTS = {
        'sender_unknown': 18,      # 모르는 번호 (피해 80%)
        'sender_shortcode': 10,    # 080/1588 등 (중위험)
        'not_in_contacts': 15,     # 연락처 미등록 (피해 4.2배)
        'first_contact': 10,       # 첫 연락 (78% 사기)
        'contains_url': 12,        # URL 포함 (67%)
        'contains_phone': 7,       # 전화번호 포함
        'financial_keywords_high': 10,  # 금융 키워드 2개+ (92%)
        'financial_keywords_low': 5,    # 금융 키워드 1개
        'urgency_keywords_high': 6,     # 긴급 키워드 2개+
        'urgency_keywords_low': 3,      # 긴급 키워드 1개
    }
    
    MAX_SCORE = 70  # 메타 스코어 최대값
    
    # 키워드 사전
    FINANCIAL_KEYWORDS = ['계좌', '이체', '입금', '송금', '대출', '카드', '결제', '은행', '금융', '출금', '돈']
    URGENCY_KEYWORDS = ['급히', '즉시', '바로', '지금', '빨리', '긴급', '당장', '서둘러', '급하게']
    
    def __init__(self):
        self.url_pattern = re.compile(r'http[s]?://|www\.|bit\.ly|\.[a-z]{2,3}/')
        self.phone_pattern = re.compile(r'010[-\s]?\d{4}[-\s]?\d{4}|080[-\s]?\d{3,4}[-\s]?\d{4}|1588[-\s]?\d{4}|1544[-\s]?\d{4}')
    
    def calculate_score(self, text: str, meta: Dict = None) -> Dict:
        """
        메타 정보 기반 위험도 점수 계산 (70점 만점)
        """
        if meta is None:
            meta = {}
        
        score = 0
        breakdown = {}
        
        # 1. 발신자 유형 (최대 18점)
        sender_type = meta.get('sender_type', 'unknown')
        if sender_type == 'unknown':
            score += self.WEIGHTS['sender_unknown']
            breakdown['발신자유형'] = self.WEIGHTS['sender_unknown']
        elif sender_type == 'shortcode':
            score += self.WEIGHTS['sender_shortcode']
            breakdown['발신자유형'] = self.WEIGHTS['sender_shortcode']
        else:
            breakdown['발신자유형'] = 0
        
        # 2. 연락처 등록 여부 (최대 15점)
        if not meta.get('in_contacts', False):
            score += self.WEIGHTS['not_in_contacts']
            breakdown['연락처등록'] = self.WEIGHTS['not_in_contacts']
        else:
            breakdown['연락처등록'] = 0
        
        # 3. 첫 연락 여부 (최대 10점)
        if meta.get('is_first_contact', True):
            score += self.WEIGHTS['first_contact']
            breakdown['첫연락'] = self.WEIGHTS['first_contact']
        else:
            breakdown['첫연락'] = 0
        
        # 4. URL 포함 여부 (최대 12점)
        if self.url_pattern.search(text):
            score += self.WEIGHTS['contains_url']
            breakdown['URL포함'] = self.WEIGHTS['contains_url']
        else:
            breakdown['URL포함'] = 0
        
        # 5. 전화번호 포함 여부 (최대 7점)
        if self.phone_pattern.search(text):
            score += self.WEIGHTS['contains_phone']
            breakdown['전화번호'] = self.WEIGHTS['contains_phone']
        else:
            breakdown['전화번호'] = 0
        
        # 6. 금융 키워드 (최대 10점)
        financial_count = sum(1 for kw in self.FINANCIAL_KEYWORDS if kw in text)
        if financial_count >= 2:
            score += self.WEIGHTS['financial_keywords_high']
            breakdown['금융키워드'] = self.WEIGHTS['financial_keywords_high']
        elif financial_count == 1:
            score += self.WEIGHTS['financial_keywords_low']
            breakdown['금융키워드'] = self.WEIGHTS['financial_keywords_low']
        else:
            breakdown['금융키워드'] = 0
        
        # 7. 긴급성 키워드 (최대 6점)
        urgency_count = sum(1 for kw in self.URGENCY_KEYWORDS if kw in text)
        if urgency_count >= 2:
            score += self.WEIGHTS['urgency_keywords_high']
            breakdown['긴급키워드'] = self.WEIGHTS['urgency_keywords_high']
        elif urgency_count == 1:
            score += self.WEIGHTS['urgency_keywords_low']
            breakdown['긴급키워드'] = self.WEIGHTS['urgency_keywords_low']
        else:
            breakdown['긴급키워드'] = 0
        
        # 최대 70점으로 제한
        score = min(score, self.MAX_SCORE)
        
        return {
            'total_score': score,
            'max_score': self.MAX_SCORE,
            'breakdown': breakdown
        }

meta_scorer = MetaScorer()
print('✅ 메타 스코어링 시스템 초기화 완료')
print(f'   최대 메타 스코어: {meta_scorer.MAX_SCORE}점 (전체 100점 중)')

In [ ]:
# 메타 스코어링 테스트
test_text = "엄마 나 폰 고장나서 급히 돈 좀 보내줘 http://bit.ly/xxx"
test_meta = {'sender_type': 'unknown', 'in_contacts': False, 'is_first_contact': True}

result = meta_scorer.calculate_score(test_text, test_meta)
print(f'테스트 메시지: {test_text}')
print(f'메타 스코어: {result["total_score"]}/{result["max_score"]} (70점 만점)')
print('\n항목별 점수:')
for key, val in result['breakdown'].items():
    if val > 0:
        print(f'  • {key}: +{val}점')

---
## 3. 데이터 로딩 및 전처리

In [ ]:
class DataPreprocessor:
    """피싱 데이터 전처리기"""
    
    @staticmethod
    def clean_text(text: str) -> str:
        if pd.isna(text):
            return ''
        text = str(text)
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    @staticmethod
    def clean_voice_transcript(text: str) -> str:
        text = DataPreprocessor.clean_text(text)
        fillers = ['어~', '음~', '아~', '에~', '어', '음', '아', '에', '뭐', '그', '저', '이제']
        for filler in fillers:
            text = text.replace(f' {filler} ', ' ')
        text = re.sub(r'OOO+', '', text)
        return re.sub(r'\s+', ' ', text).strip()
    
    @staticmethod
    def truncate_long_text(text: str, max_words: int = 100) -> str:
        words = text.split()
        if len(words) > max_words:
            return ' '.join(words[:max_words])
        return text

preprocessor = DataPreprocessor()
print('✅ 전처리기 초기화 완료')

In [ ]:
# 데이터셋 1: KOR_phishing (메신저 피싱)
df1 = pd.read_csv('KOR_phishing_data_raw.csv', encoding='utf-8')
df1 = df1.dropna(subset=['content', 'class'])
df1['text'] = df1['content'].apply(preprocessor.clean_text)
df1['label'] = df1['class'].astype(int)
df1['source'] = 'KOR_phishing'

print('=== 데이터셋 1: KOR_phishing ===')
print(f'총: {len(df1):,}건')
print(f'피싱: {sum(df1["label"]==1):,}건')
print(f'정상: {sum(df1["label"]==0):,}건')

In [ ]:
# 데이터셋 2: KorCCVi (보이스피싱)
df2 = pd.read_csv('KorCCVi_v2.1.csv', encoding='utf-8')
df2 = df2.dropna(subset=['transcript', 'label'])
df2['text'] = df2['transcript'].apply(preprocessor.clean_voice_transcript)
df2['text'] = df2['text'].apply(lambda x: preprocessor.truncate_long_text(x, max_words=100))
df2['label'] = df2['label'].astype(int)
df2['source'] = 'KorCCVi'

print('=== 데이터셋 2: KorCCVi ===')
print(f'총: {len(df2):,}건')
print(f'피싱: {sum(df2["label"]==1):,}건')
print(f'정상: {sum(df2["label"]==0):,}건')

In [ ]:
# 데이터 통합
df_combined = pd.concat([
    df1[['text', 'label', 'source']],
    df2[['text', 'label', 'source']]
], ignore_index=True)

df_combined = df_combined[df_combined['text'].str.len() > 10]

print('=== 통합 데이터셋 ===')
print(f'총: {len(df_combined):,}건')
print(f'피싱: {sum(df_combined["label"]==1):,}건 ({100*sum(df_combined["label"]==1)/len(df_combined):.1f}%)')
print(f'정상: {sum(df_combined["label"]==0):,}건')

In [ ]:
# 데이터 분할
train_df, temp_df = train_test_split(
    df_combined[['text', 'label', 'source']],
    test_size=0.2, random_state=42, stratify=df_combined['label']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label']
)

print('=== 데이터 분할 ===')
print(f'Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}')

---
## 4. KoELECTRA 모델

In [ ]:
MODEL_NAME = 'monologg/koelectra-small-v3-discriminator'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2,
    id2label={0: 'normal', 1: 'phishing'},
    label2id={'normal': 0, 'phishing': 1}
).to(device)

print(f'✅ 모델: {MODEL_NAME}')
print(f'   파라미터: {model.num_parameters():,}')

In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

train_ds = HFDataset.from_pandas(train_df[['text', 'label']].rename(columns={'label': 'labels'}).reset_index(drop=True))
val_ds = HFDataset.from_pandas(val_df[['text', 'label']].rename(columns={'label': 'labels'}).reset_index(drop=True))
test_ds = HFDataset.from_pandas(test_df[['text', 'label']].rename(columns={'label': 'labels'}).reset_index(drop=True))

train_ds = train_ds.map(tokenize_fn, batched=True)
val_ds = val_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

print('✅ 토큰화 완료')

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_df['label'].values)
print(f'클래스 가중치: 정상={class_weights[0]:.2f}, 피싱={class_weights[1]:.2f}')

class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop('labels')
        outputs = model(**inputs)
        loss_fn = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fn(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

training_args = TrainingArguments(
    output_dir='./scamshield_v8_1',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=3e-5,
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to='none',
)

trainer = WeightedTrainer(
    class_weights=class_weights,
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print('✅ 학습 준비 완료')

In [ ]:
print('학습 시작...')
trainer.train()
print('✅ 학습 완료!')

---
## 5. 통합 탐지기 (100점 만점)

In [ ]:
class ScamShieldV81:
    """
    ScamShield v8.1 통합 피싱 탐지기
    
    점수 체계 (100점 만점):
    - AI 점수: 0-30점 (피싱 확률 × 30)
    - 메타 스코어: 0-70점
    
    위험 등급:
    - 🟢 안전: 0-24점
    - 🟡 주의: 25-49점
    - 🟠 위험: 50-74점
    - 🔴 긴급: 75점 이상
    """
    
    AI_MAX_SCORE = 30
    META_MAX_SCORE = 70
    TOTAL_MAX_SCORE = 100
    
    GRADE_THRESHOLDS = [
        (75, '🔴 긴급', 'CRITICAL'),
        (50, '🟠 위험', 'DANGER'),
        (25, '🟡 주의', 'WARNING'),
        (0, '🟢 안전', 'SAFE'),
    ]
    
    def __init__(self, trainer, tokenizer, meta_scorer):
        self.model = trainer.model
        self.tokenizer = tokenizer
        self.meta_scorer = meta_scorer
        self.device = next(self.model.parameters()).device
        self.model.eval()
    
    def analyze(self, text: str, meta: Dict = None) -> Dict:
        # 1. AI 모델 점수 (0-30점)
        inputs = self.tokenizer(
            text, return_tensors='pt', truncation=True, max_length=256, padding=True
        ).to(self.device)
        
        with torch.no_grad():
            probs = torch.softmax(self.model(**inputs).logits, dim=1)
        
        ai_prob = probs[0][1].item()
        ai_score = int(ai_prob * self.AI_MAX_SCORE)
        
        # 2. 메타 스코어 (0-70점)
        meta_result = self.meta_scorer.calculate_score(text, meta)
        meta_score = meta_result['total_score']
        
        # 3. 최종 점수 (100점 만점)
        final_score = min(ai_score + meta_score, self.TOTAL_MAX_SCORE)
        grade_emoji, grade_name = self._get_grade(final_score)
        
        return {
            'text': text[:60] + '...' if len(text) > 60 else text,
            'ai_score': ai_score,
            'ai_max': self.AI_MAX_SCORE,
            'ai_prob': f'{ai_prob*100:.1f}%',
            'meta_score': meta_score,
            'meta_max': self.META_MAX_SCORE,
            'meta_breakdown': meta_result['breakdown'],
            'final_score': final_score,
            'max_score': self.TOTAL_MAX_SCORE,
            'grade': grade_emoji,
            'grade_name': grade_name,
            'is_dangerous': final_score >= 50
        }
    
    def _get_grade(self, score: int) -> tuple:
        for threshold, emoji, name in self.GRADE_THRESHOLDS:
            if score >= threshold:
                return emoji, name
        return '🟢 안전', 'SAFE'

detector = ScamShieldV81(trainer, tokenizer, meta_scorer)
print('✅ ScamShield v8.1 통합 탐지기 초기화 완료')
print(f'   점수 체계: AI {detector.AI_MAX_SCORE}점 + 메타 {detector.META_MAX_SCORE}점 = {detector.TOTAL_MAX_SCORE}점 만점')

---
## 6. Multi-Guardian 보호자 알림 시스템

In [ ]:
class MultiGuardianSystem:
    """다중 보호자 알림 시스템"""
    
    def __init__(self):
        self.guardians = []
        self.alert_history = []
    
    def add_guardian(self, name: str, phone: str, priority: int = 1):
        self.guardians.append({'name': name, 'phone': phone, 'priority': priority})
        self.guardians.sort(key=lambda x: x['priority'])
    
    def send_alert(self, result: Dict) -> Optional[Dict]:
        if result['final_score'] < 50:
            return None
        
        if result['final_score'] >= 75:
            alert_type = '🚨 긴급'
            recipients = self.guardians[:1]
        else:
            alert_type = '⚠️ 위험'
            recipients = self.guardians
        
        alert = {
            'type': alert_type,
            'score': result['final_score'],
            'grade': result['grade'],
            'message': result['text'],
            'recipients': [g['name'] for g in recipients],
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        self.alert_history.append(alert)
        return alert

guardian_system = MultiGuardianSystem()
guardian_system.add_guardian('아들 (철수)', '010-1234-5678', priority=1)
guardian_system.add_guardian('딸 (영희)', '010-9876-5432', priority=2)

print('✅ Multi-Guardian 시스템 초기화 완료')

---
## 7. 데모 및 시각화

In [ ]:
# 테스트 케이스 (경계 케이스 포함)
test_cases = [
    {
        'text': '엄마 나 폰 고장나서 급히 돈 좀 보내줘 http://bit.ly/xxx',
        'meta': {'sender_type': 'unknown', 'in_contacts': False, 'is_first_contact': True},
        'expected': '긴급'
    },
    {
        'text': '대검찰청입니다. 계좌 확인 필요합니다. 080-123-4567',
        'meta': {'sender_type': 'shortcode', 'in_contacts': False, 'is_first_contact': True},
        'expected': '위험/긴급'
    },
    {
        'text': '택배 배송 지연 안내입니다.',
        'meta': {'sender_type': 'shortcode', 'in_contacts': False, 'is_first_contact': True},
        'expected': '주의'
    },
    {
        'text': '엄마 오늘 저녁 뭐 먹을까?',
        'meta': {'sender_type': 'registered', 'in_contacts': True, 'is_first_contact': False},
        'expected': '안전'
    },
    {
        'text': '회의 시간이 3시로 변경되었습니다.',
        'meta': {'sender_type': 'registered', 'in_contacts': True, 'is_first_contact': False},
        'expected': '안전'
    },
    {
        'text': '엄마 나 급히 돈 좀 보내줘',
        'meta': {'sender_type': 'registered', 'in_contacts': True, 'is_first_contact': False},
        'expected': '안전/주의'
    },
]

print('=' * 70)
print('🛡️ ScamShield Family v8.1 Demo (100점 만점)')
print('=' * 70)

results = []
for i, case in enumerate(test_cases, 1):
    result = detector.analyze(case['text'], case['meta'])
    results.append(result)
    
    print(f"\n[{i}] {result['grade']} ({result['final_score']}/{result['max_score']}점)")
    print(f"    메시지: {result['text']}")
    print(f"    AI: {result['ai_score']}/{result['ai_max']}점 | 메타: {result['meta_score']}/{result['meta_max']}점")
    
    alert = guardian_system.send_alert(result)
    if alert:
        print(f"    ➡️ 보호자 알림: {alert['recipients']}")

In [ ]:
# 검증 결과 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 위험 등급별 분포
grade_counts = {'안전': 0, '주의': 0, '위험': 0, '긴급': 0}
for r in results:
    if r['final_score'] >= 75:
        grade_counts['긴급'] += 1
    elif r['final_score'] >= 50:
        grade_counts['위험'] += 1
    elif r['final_score'] >= 25:
        grade_counts['주의'] += 1
    else:
        grade_counts['안전'] += 1

colors = ['#2ecc71', '#f1c40f', '#e67e22', '#e74c3c']
axes[0, 0].pie(list(grade_counts.values()), labels=list(grade_counts.keys()),
               colors=colors, autopct='%1.0f%%', startangle=90)
axes[0, 0].set_title('위험 등급별 분포', fontsize=14)

# 2. AI vs 메타 스코어
ai_scores = [r['ai_score'] for r in results]
meta_scores = [r['meta_score'] for r in results]
final_scores = [r['final_score'] for r in results]

scatter = axes[0, 1].scatter(ai_scores, meta_scores, c=final_scores, cmap='RdYlGn_r', s=200, edgecolors='black', vmin=0, vmax=100)
axes[0, 1].set_xlabel('AI 모델 점수 (0-30)', fontsize=12)
axes[0, 1].set_ylabel('메타 스코어 (0-70)', fontsize=12)
axes[0, 1].set_title('AI vs 메타 스코어', fontsize=14)
axes[0, 1].set_xlim(-2, 35)
axes[0, 1].set_ylim(-5, 75)
plt.colorbar(scatter, ax=axes[0, 1], label='최종 점수 (100점 만점)')

# 3. 피처 가중치 (100점 기준)
feature_importance = {
    '발신자유형': 18, '연락처등록': 15, 'URL포함': 12,
    '첫연락': 10, '금융키워드': 10, '전화번호': 7, '긴급키워드': 6
}
bars = axes[1, 0].barh(list(feature_importance.keys()), list(feature_importance.values()), color='#3498db')
axes[1, 0].set_xlabel('가중치 (점수)', fontsize=12)
axes[1, 0].set_title('메타 피처 가중치 (70점 만점 중)', fontsize=14)
for bar, val in zip(bars, feature_importance.values()):
    axes[1, 0].text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2, f'{val}점', va='center')

# 4. 테스트 결과
test_labels = [f'테스트 {i+1}' for i in range(len(results))]
bar_colors = ['#e74c3c' if r['final_score'] >= 75 else '#e67e22' if r['final_score'] >= 50 else '#f1c40f' if r['final_score'] >= 25 else '#2ecc71' for r in results]

axes[1, 1].bar(test_labels, final_scores, color=bar_colors, edgecolor='black')
axes[1, 1].axhline(y=75, color='#e74c3c', linestyle='--', label='긴급 (75)')
axes[1, 1].axhline(y=50, color='#e67e22', linestyle='--', label='위험 (50)')
axes[1, 1].axhline(y=25, color='#f1c40f', linestyle='--', label='주의 (25)')
axes[1, 1].set_ylabel('최종 점수 (100점 만점)', fontsize=12)
axes[1, 1].set_title('테스트 케이스별 점수', fontsize=14)
axes[1, 1].set_ylim(0, 105)
axes[1, 1].legend(loc='upper right')

plt.tight_layout()
plt.savefig('evaluation_v8_1.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n✅ 시각화 저장: evaluation_v8_1.png')

---
## 8. Summary

### v8.1 점수 체계 (100점 만점)

| 구분 | 점수 | 비중 |
|------|------|------|
| AI 모델 | 0-30점 | 30% |
| 메타 스코어 | 0-70점 | 70% |
| **합계** | **100점** | **100%** |

### 메타 스코어 가중치 (통계 기반)

| 항목 | 점수 | 근거 |
|------|------|------|
| 모르는 번호 | 18점 | 피싱 80% 미등록 발신 |
| 연락처 미등록 | 15점 | 미등록 시 피해 4.2배 |
| URL 포함 | 12점 | 스팸 67% URL 포함 |
| 첫 연락 | 10점 | 첫 연락 사기 78% |
| 금융 키워드 | 10점 | 피싱 92% 금융키워드 |
| 전화번호 포함 | 7점 | 콜백 유도 43% |
| 긴급 키워드 | 6점 | 긴급성 호소 |

### 위험 등급
- 🔴 긴급: 75점 이상
- 🟠 위험: 50-74점
- 🟡 주의: 25-49점
- 🟢 안전: 0-24점

### 참고 자료
- 금융감독원 보이스피싱 피해현황 (2023)
- KISA 스팸 동향 분석 보고서 (2023)
- Kim et al. (2024) "Detection of Korean Phishing Messages"

---
*ScamShield Family v8.1 - 100점 만점 통합 스코어링*